<a id='top'></a>

# Data Engineering of StatsBomb Data
##### Notebook to engineer previous parsed JSON data from the [StatsBomb Open Data GitHub repository](https://github.com/statsbomb/open-data)

### By [Edd Webster](https://www.twitter.com/eddwebster)
Notebook first written: 10/11/2020<br>
Notebook last updated: 18/02/2021

![title](../../img/logos/stats-bomb-logo.png)

Click [here](#section5) to jump straight to the Exploratory Data Analysis section and skip the [Task Brief](#section2), [Data Sources](#section3), and [Data Engineering](#section4) sections. Or click [here](#section6) to jump straight to the Conclusion.

___


## <a id='import_libraries'>Introduction</a>
This notebook parses pubicly available [StatsBomb](https://statsbomb.com/) Event data, using [pandas](http://pandas.pydata.org/) for data manipulation through DataFrames.

For more information about this notebook and the author, I'm available through all the following channels:
*    [eddwebster.com](https://www.eddwebster.com/);
*    edd.j.webster@gmail.com;
*    [@eddwebster](https://www.twitter.com/eddwebster);
*    [linkedin.com/in/eddwebster](https://www.linkedin.com/in/eddwebster/);
*    [github/eddwebster](https://github.com/eddwebster/);
*    [public.tableau.com/profile/edd.webster](https://public.tableau.com/profile/edd.webster);
*    [kaggle.com/eddwebster](https://www.kaggle.com/eddwebster); and
*    [hackerrank.com/eddwebster](https://www.hackerrank.com/eddwebster).

![title](../../img/edd_webster/fifa21eddwebsterbanner.png)

The accompanying GitHub repository for this notebook can be found [here](https://github.com/eddwebster/football_analytics) and a static version of this notebook can be found [here](https://nbviewer.jupyter.org/github/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/StatsBomb%20Data%20Engineering.ipynb).

___

## <a id='notebook_contents'>Notebook Contents</a>
1.    [Notebook Dependencies](#section1)<br>
2.    [Project Brief](#section2)<br>
3.    [Data Sources](#section3)<br>
      1.    [Introduction](#section3.1)<br>
      2.    [Download the Data](#section3.2)<br>
      3.    [Read in the Datasets](#section3.3)<br>
      4.    [Join the Datasets](#section3.4)<br>
      5.    [Initial Data Handling](#section3.5)<br>
4.    [Data Engineering](#section4)<br>
      1.    [Assign Raw DataFrame to Engineered DataFrame](#section4.1)<br>
      2.    [Sort the DataFrame](#section4.2)<br>
      3.    [Create Sort the DataFrame](#section4.3)<br>
      4.    [Subset DataFrame](#section4.4)<br>
5.    [Export DataFrame](#section5)<br>
6.    [Summary](#section6)<br>
7.    [Next Steps](#section7)<br>
8.    [Bibliography](#section8)<br>

___

<a id='section1'></a>

## <a id='#section1'>1. Notebook Dependencies</a>

This notebook was written using [Python 3](https://docs.python.org/3.7/) and requires the following libraries:
*    [`Jupyter notebooks`](https://jupyter.org/) for this notebook environment with which this project is presented;
*    [`NumPy`](http://www.numpy.org/) for multidimensional array computing;
*    [`pandas`](http://pandas.pydata.org/) for data analysis and manipulation; and
*    `tqdm` for a clean progress bar;

All packages used for this notebook except for BeautifulSoup can be obtained by downloading and installing the [Conda](https://anaconda.org/anaconda/conda) distribution, available on all platforms (Windows, Linux and Mac OSX). Step-by-step guides on how to install Anaconda can be found for Windows [here](https://medium.com/@GalarnykMichael/install-python-on-windows-anaconda-c63c7c3d1444) and Mac [here](https://medium.com/@GalarnykMichael/install-python-on-mac-anaconda-ccd9f2014072), as well as in the Anaconda documentation itself [here](https://docs.anaconda.com/anaconda/install/).

### Import Libraries and Modules

In [3]:
%load_ext autoreload
%autoreload 2

# Python ≥3.5 (ideally)
import platform
import sys, getopt
assert sys.version_info >= (3, 5)
import csv

# Import Dependencies
%matplotlib inline

# Math Operations
import numpy as np
from math import pi

# Datetime
import datetime
from datetime import date
import time

# Data Preprocessing
import pandas as pd    # version 1.0.3
import os    #  used to read the csv filenames
import re
import random
from io import BytesIO
from pathlib import Path

# Reading directories
import glob
import os

# Working with JSON
import json
import codecs
from pandas.io.json import json_normalize

# Football Libraries
from FCPython import createPitch

# Data Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')
import missingno as msno    # visually display missing data

# Progress Bar
from tqdm import tqdm    # a clean progress bar library

# Display in Jupyter
from IPython.display import Image, Video, YouTubeVideo
from IPython.core.display import HTML

# Ignore Warnings
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

print('Setup Complete')

Setup Complete


In [4]:
# Python / module versions used here for reference
print('Python: {}'.format(platform.python_version()))
print('NumPy: {}'.format(np.__version__))
print('pandas: {}'.format(pd.__version__))
print('matplotlib: {}'.format(mpl.__version__))
print('Seaborn: {}'.format(sns.__version__))

Python: 3.7.6
NumPy: 1.18.0
pandas: 1.2.0
matplotlib: 3.3.2
Seaborn: 0.11.1


### Defined Variables

In [5]:
# Define today's date
today = datetime.datetime.now().strftime('%d/%m/%Y').replace('/', '')

### Defined Filepaths

In [6]:
# Set up initial paths to subfolders
base_dir = os.path.join('..', '..')
data_dir = os.path.join(base_dir, 'data')
data_dir_sb = os.path.join(base_dir, 'data', 'sb')
scripts_dir = os.path.join(base_dir, 'scripts')
scripts_dir_sb = os.path.join(base_dir, 'scripts', 'sb')
data_dir_understat = os.path.join(base_dir, 'data', 'understat')
img_dir = os.path.join(base_dir, 'img')
fig_dir = os.path.join(base_dir, 'img', 'fig')
video_dir = os.path.join(base_dir, 'video')

### Custom Functions

In [7]:
# Define custom function to read JSON files that also handles the encoding of special characters e.g. accents in names of players and teams
def read_json_file(filename):
    with open(filename, 'rb') as json_file:
        return BytesIO(json_file.read()).getvalue().decode('unicode_escape')
    
# Define custom function to flatten pandas DataFrames with nested JSON columns. Source: https://stackoverflow.com/questions/39899005/how-to-flatten-a-pandas-dataframe-with-some-columns-as-json
def flatten_nested_json_df(df):

    df = df.reset_index()

    print(f"original shape: {df.shape}")
    print(f"original columns: {df.columns}")


    # search for columns to explode/flatten
    s = (df.applymap(type) == list).all()
    list_columns = s[s].index.tolist()

    s = (df.applymap(type) == dict).all()
    dict_columns = s[s].index.tolist()

    print(f"lists: {list_columns}, dicts: {dict_columns}")
    while len(list_columns) > 0 or len(dict_columns) > 0:
        new_columns = []

        for col in dict_columns:
            print(f"flattening: {col}")
            # explode dictionaries horizontally, adding new columns
            horiz_exploded = pd.json_normalize(df[col]).add_prefix(f'{col}.')
            horiz_exploded.index = df.index
            df = pd.concat([df, horiz_exploded], axis=1).drop(columns=[col])
            new_columns.extend(horiz_exploded.columns) # inplace

        for col in list_columns:
            print(f"exploding: {col}")
            # explode lists vertically, adding new columns
            df = df.drop(columns=[col]).join(df[col].explode().to_frame())
            new_columns.append(col)

        # check if there are still dict o list fields to flatten
        s = (df[new_columns].applymap(type) == list).all()
        list_columns = s[s].index.tolist()

        s = (df[new_columns].applymap(type) == dict).all()
        dict_columns = s[s].index.tolist()

        print(f"lists: {list_columns}, dicts: {dict_columns}")

    print(f"final shape: {df.shape}")
    print(f"final columns: {df.columns}")
    return df

### Notebook Settings

In [8]:
pd.set_option('display.max_columns', None)

---

<a id='section2'></a>

## <a id='#section2'>2. Project Brief</a>
This Jupyter notebook explores how to parse publicly available Event data from [StatsBomb](https://statsbomb.com/) using [pandas](http://pandas.pydata.org/) for data manipulation through DataFrames.


The combined event data roduced in this notebook is exported to CSV. This data can be further analysed in Python, joined to other datasets, or explored using Tableau, PowerBI, Microsoft Excel.


**Notebook Conventions**:<br>
*    Variables that refer a `DataFrame` object are prefixed with `df_`.
*    Variables that refer to a collection of `DataFrame` objects (e.g., a list, a set or a dict) are prefixed with `dfs_`.

---

## <a id='#section3'>3. Data Sources</a>

### <a id='#section3.1'>3.1. Introduction</a>

#### <a id='#section3.1.1'>3.1.1. About StatsBomb</a>
[StatsBomb](https://statsbomb.com/) are a football analytics and data company.

![title](../../img/logos/stats-bomb-logo.png)

Before conducting our EDA, the data needs to be imported as a DataFrame in the Data Sources section [Section 3](#section3) and Cleaned in the Data Engineering section [Section 4](#section4).

We'll be using the [pandas](http://pandas.pydata.org/) library to import our data to this workbook as a DataFrame.

#### <a id='#section3.1.2'>3.1.2. About the StatsBomb publicly available data</a>
The complete data set contains:
- 7 competitions;
- 879 matches;
- 3,161,917 events; and
- z players.

The datasets we will be using are:
- competitions;
- matches;
- events;
- lineups; and
- tactics;

The data needs to be imported as a DataFrame in the Data Sources section [Section 3](#section3) and cleaned in the Data Engineering section [Section 4](#section4).

### <a id='#section3.2'>3.2. Read in Data</a>

#### <a id='#section3.2.1'>3.2.1. Data dictionary

#### <a id='#section3.2.2'>3.2.2. Read in data

In [9]:
# Show files in directory
print(glob.glob(os.path.join(data_dir_sb, 'combined', 'raw', 'csv')))

['../../data/sb/combined/raw/csv']


In [10]:
# Read CSV file as a pandas DataFrame
df_sb = pd.read_csv(os.path.join(data_dir_sb, 'combined', 'raw', 'csv', 'combined.csv'))

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (18,40,42,50,51,53,54,77,78,79,80,81,84,85,86,87,89,91,93,96,97,98,99,100,101,102,103,104,106,107,108,109,110,111,112,113,114,116,118,120,121,122,123,125,127,128,129,130,131,132,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,159,160,165,169,170,176,177,190) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### <a id='#section3.3'>3.3. Initial Data Handling</a>

In [11]:
# Display the first 5 rows of the raw DataFrame, df_sb
df_sb.head()

,level_0,id,index_x,period,timestamp,minute,second,possession,duration,type_id,type_name,possession_team_id,possession_team_name,play_pattern_id,play_pattern_name,team_id,team_name,tactics_formation,tactics_lineup,related_events,location,player_id,player_name,position_id,position_name,pass_recipient_id,pass_recipient_name,pass_length,pass_angle,pass_height_id,pass_height_name,pass_end_location,pass_type_id,pass_type_name,pass_body_part_id,pass_body_part_name,carry_end_location,pass_outcome_id,pass_outcome_name,under_pressure,clearance_head,clearance_body_part_id,clearance_body_part_name,counterpress,duel_outcome_id,duel_outcome_name,duel_type_id,duel_type_name,ball_receipt_outcome_id,ball_receipt_outcome_name,out,clearance_left_foot,pass_switch,off_camera,clearance_aerial_won,dribble_outcome_id,dribble_outcome_name,pass_cross,pass_assisted_shot_id,pass_shot_assist,shot_statsbomb_xg,shot_end_location,shot_key_pass_id,shot_body_part_id,shot_body_part_name,shot_technique_id,shot_technique_name,shot_outcome_id,shot_outcome_name,shot_type_id,shot_type_name,shot_freeze_frame,goalkeeper_end_location,goalkeeper_type_id,goalkeeper_type_name,goalkeeper_position_id,goalkeeper_position_name,ball_recovery_recovery_failure,foul_committed_advantage,foul_won_advantage,dribble_overrun,clearance_right_foot,interception_outcome_id,interception_outcome_name,foul_won_defensive,pass_aerial_won,pass_deflected,pass_inswinging,pass_technique_id,pass_technique_name,goalkeeper_body_part_id,goalkeeper_body_part_name,goalkeeper_technique_id,goalkeeper_technique_name,goalkeeper_outcome_id,goalkeeper_outcome_name,pass_outswinging,pass_goal_assist,shot_one_on_one,miscontrol_aerial_won,shot_deflected,block_deflection,shot_first_time,block_offensive,pass_through_ball,foul_committed_card_id,foul_committed_card_name,foul_committed_penalty,foul_won_penalty,dribble_nutmeg,pass_miscommunication,pass_no_touch,foul_committed_offensive,goalkeeper_lost_out,pass_straight,substitution_outcome_id,substitution_outcome_name,substitution_replacement_id,substitution_replacement_name,match_id,goalkeeper_punched_out,shot_aerial_won,pass_cut_back,goalkeeper_success_in_play,50_50_outcome_id,50_50_outcome_name,foul_committed_type_id,foul_committed_type_name,ball_recovery_offensive,shot_saved_off_target,goalkeeper_shot_saved_off_target,shot_open_goal,dribble_no_touch,bad_behaviour_card_id,bad_behaviour_card_name,half_start_late_video_start,block_save_block,shot_follows_dribble,clearance_other,goalkeeper_shot_saved_to_post,shot_redirect,injury_stoppage_in_chain,shot_saved_to_post,goalkeeper_success_out,goalkeeper_lost_in_play,half_end_early_video_end,player_off_permanent,goalkeeper_saved_to_post,pass_backheel,shot_kick_off,goalkeeper_penalty_saved_to_post,index_y,match_date,kick_off,home_score,away_score,match_status,last_updated,match_week,referee,stadium,competition_competition_id,competition_country_name,competition_competition_name,season_season_id,season_season_name,home_team_home_team_id,home_team_home_team_name,home_team_home_team_gender,home_team_home_team_group,home_team_managers,home_team_country_id,home_team_country_name,away_team_away_team_id,away_team_away_team_name,away_team_away_team_gender,away_team_away_team_group,away_team_managers,away_team_country_id,away_team_country_name,metadata_data_version,metadata_shot_fidelity_version,metadata_xy_fidelity_version,competition_stage_id,competition_stage_name,competition_id,season_id,country_name,competition_name,competition_gender,season_name,match_updated,match_available
0,0,41e0ff39-da7c-451a-8f08-82d3a9b369f2,1,1,00:00:00.000,0,0,1,0.000000,35,Starting XI,1,Arsenal,1,Regular Play,1,Arsenal,442.0,"[{'player': {'id': 20015, 'name': 'Jens Lehman...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [12]:
# Display the last 5 rows of the raw DataFrame, df_sb
df_sb.tail()

,level_0,id,index_x,period,timestamp,minute,second,possession,duration,type_id,type_name,possession_team_id,possession_team_name,play_pattern_id,play_pattern_name,team_id,team_name,tactics_formation,tactics_lineup,related_events,location,player_id,player_name,position_id,position_name,pass_recipient_id,pass_recipient_name,pass_length,pass_angle,pass_height_id,pass_height_name,pass_end_location,pass_type_id,pass_type_name,pass_body_part_id,pass_body_part_name,carry_end_location,pass_outcome_id,pass_outcome_name,under_pressure,clearance_head,clearance_body_part_id,clearance_body_part_name,counterpress,duel_outcome_id,duel_outcome_name,duel_type_id,duel_type_name,ball_receipt_outcome_id,ball_receipt_outcome_name,out,clearance_left_foot,pass_switch,off_camera,clearance_aerial_won,dribble_outcome_id,dribble_outcome_name,pass_cross,pass_assisted_shot_id,pass_shot_assist,shot_statsbomb_xg,shot_end_location,shot_key_pass_id,shot_body_part_id,shot_body_part_name,shot_technique_id,shot_technique_name,shot_outcome_id,shot_outcome_name,shot_type_id,shot_type_name,shot_freeze_frame,goalkeeper_end_location,goalkeeper_type_id,goalkeeper_type_name,goalkeeper_position_id,goalkeeper_position_name,ball_recovery_recovery_failure,foul_committed_advantage,foul_won_advantage,dribble_overrun,clearance_right_foot,interception_outcome_id,interception_outcome_name,foul_won_defensive,pass_aerial_won,pass_deflected,pass_inswinging,pass_technique_id,pass_technique_name,goalkeeper_body_part_id,goalkeeper_body_part_name,goalkeeper_technique_id,goalkeeper_technique_name,goalkeeper_outcome_id,goalkeeper_outcome_name,pass_outswinging,pass_goal_assist,shot_one_on_one,miscontrol_aerial_won,shot_deflected,block_deflection,shot_first_time,block_offensive,pass_through_ball,foul_committed_card_id,foul_committed_card_name,foul_committed_penalty,foul_won_penalty,dribble_nutmeg,pass_miscommunication,pass_no_touch,foul_committed_offensive,goalkeeper_lost_out,pass_straight,substitution_outcome_id,substitution_outcome_name,substitution_replacement_id,substitution_replacement_name,match_id,goalkeeper_punched_out,shot_aerial_won,pass_cut_back,goalkeeper_success_in_play,50_50_outcome_id,50_50_outcome_name,foul_committed_type_id,foul_committed_type_name,ball_recovery_offensive,shot_saved_off_target,goalkeeper_shot_saved_off_target,shot_open_goal,dribble_no_touch,bad_behaviour_card_id,bad_behaviour_card_name,half_start_late_video_start,block_save_block,shot_follows_dribble,clearance_other,goalkeeper_shot_saved_to_post,shot_redirect,injury_stoppage_in_chain,shot_saved_to_post,goalkeeper_success_out,goalkeeper_lost_in_play,half_end_early_video_end,player_off_permanent,goalkeeper_saved_to_post,pass_backheel,shot_kick_off,goalkeeper_penalty_saved_to_post,index_y,match_date,kick_off,home_score,away_score,match_status,last_updated,match_week,referee,stadium,competition_competition_id,competition_country_name,competition_competition_name,season_season_id,season_season_name,home_team_home_team_id,home_team_home_team_name,home_team_home_team_gender,home_team_home_team_group,home_team_managers,home_team_country_id,home_team_country_name,away_team_away_team_id,away_team_away_team_name,away_team_away_team_gender,away_team_away_team_group,away_team_managers,away_team_country_id,away_team_country_name,metadata_data_version,metadata_shot_fidelity_version,metadata_xy_fidelity_version,competition_stage_id,competition_stage_name,competition_id,season_id,country_name,competition_name,competition_gender,season_name,match_updated,match_available
3158153,4237,3821ac0d-f832-4bbd-a9c3-f00dc45bfd2a,4238,4,00:22:30.391,127,30,268,0.040000,43,Carry,858,Sweden Women's,4,From Throw In,858,Sweden Women's,NaN,NaN,"['4d7320fa-dde2-4191-9172-05745c4e25d2', '633c...","[106.0, 3.8]",10222.0,Jonna Andersson,6.0,Left Back,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[106.0, 3.8]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [13]:
# Print the shape of the raw DataFrame, df_sb
print(df_sb.shape)

(3158158, 193)


In [14]:
# Print the column names of the raw DataFrame, df_sb
print(df_sb.columns)

Index(['level_0', 'id', 'index_x', 'period', 'timestamp', 'minute', 'second',
       'possession', 'duration', 'type_id',
       ...
       'competition_stage_id', 'competition_stage_name', 'competition_id',
       'season_id', 'country_name', 'competition_name', 'competition_gender',
       'season_name', 'match_updated', 'match_available'],
      dtype='object', length=193)


The joined dataset has one hundred and ninety three features (columns). Full details of these attributes can be found in the [Data Dictionary](section3.2.1).

In [21]:
# Data types of the features of the raw DataFrame, df_sb
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_sb.dtypes)

level_0                               int64
id                                   object
index_x                               int64
period                                int64
timestamp                            object
minute                                int64
second                                int64
possession                            int64
duration                            float64
type_id                               int64
type_name                            object
possession_team_id                    int64
possession_team_name                 object
play_pattern_id                       int64
play_pattern_name                    object
team_id                               int64
team_name                            object
tactics_formation                   float64
tactics_lineup                       object
related_events                       object
location                             object
player_id                           float64
player_name                     

Full details of these attributes and their data types can be found in the [Data Dictionary](section3.3.1).

In [16]:
# Info for the raw DataFrame, df_sb
df_sb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3158158 entries, 0 to 3158157
Columns: 193 entries, level_0 to match_available
dtypes: float64(36), int64(24), object(133)
memory usage: 4.5+ GB


In [17]:
# Description of the raw DataFrame, df_sb, showing some summary statistics for each numberical column in the DataFrame
df_sb.describe()

,level_0,index_x,period,minute,second,possession,duration,type_id,possession_team_id,play_pattern_id,team_id,tactics_formation,player_id,position_id,pass_recipient_id,pass_length,pass_angle,pass_height_id,pass_type_id,pass_body_part_id,pass_outcome_id,clearance_body_part_id,duel_outcome_id,duel_type_id,ball_receipt_outcome_id,dribble_outcome_id,shot_statsbomb_xg,shot_body_part_id,shot_technique_id,shot_outcome_id,shot_type_id,goalkeeper_type_id,goalkeeper_position_id,interception_outcome_id,pass_technique_id,goalkeeper_body_part_id,goalkeeper_technique_id,goalkeeper_outcome_id,foul_committed_card_id,substitution_outcome_id,substitution_replacement_id,match_id,50_50_outcome_id,foul_committed_type_id,bad_behaviour_card_id,index_y,home_score,away_score,match_week,competition_competition_id,season_season_id,home_team_home_team_id,home_team_country_id,away_team_away_team_id,away_team_country_id,metadata_shot_fidelity_version,metadata_xy_fidelity_version,competition_stage_id,competition_id,season_id
count,3.158158e+06,3.158158e+06,3.158158e+06,3.158158e+06,3.158158e+06,3.158158e+06,2.313994e+06,3.158158e+06,3.158158e+06,3.158158e+06,3.158158e+06,3301.000000,3.141590e+06,3.141590e+06,809438.000000,871308.000000,871308.000000,871308.000000,171945.00000,819407.000000,185449.000000,30932.000000,34423.000000,55407.000000,123579.0,36494.000000,22357.000000,22357.000000,22357.000000,22357.000000,22357.000000,26473.000000,22337.000000,16815.000000,10304.000000,6448.000000,8660.000000,12634.000000,2799.000000,4842.000000,4847.000000,3.158158e+06,1366.000000,1575.000000,628.000000,3.158158e+06,3.158158e+06,3.158158e+06,3.158158e+06,3.158158e+06,3.158158e+06,3.158158e+06,3.158158e+06,3.158158e+06,3.158158e+06,2494494.0,2154338.0,3.158158e+06,3.158158e+06,3.158158e+06
mean,1.820498e+03,1.821498e+03,1.500939e+00,4.486634e+01,2.930872e+01,9.664262e+01,1.273816e+00,3.236920e+01,4.930352e+02,2.823449e+00,4.976638e+02,4814.252348,1.263005e+04,1.124376e+01,12222.931576,21.058518,0.013494,1.555189,65.13010,39.882497,19.264547,38.253136,11.894199,10.621275,9.0,8.381789,0.117009,38.998166,92.906070,98.163394,85.528380,31.258868,43.852532,10.267023,106.536685,36.551179,45.561085,40.609387,6.931404,102.945477,14064.650505,4.397187e+05,2.036603,23.274286,6.939490,2.397784e+01,1.808959e+00,1.436360e+00,1.491716e+01,2.319544e+01,2.205346e+01,5.127428e+02,1.671042e+02,5.100286e+02,1.666108e+02,2.0,2.0,2.563879e+00,2.319544e+01,2.205346e+01
std,1.076392e+03,1.076392e+03,5.160235e-01,2.714136e+01,1.738708e+01,5.767026e+01,2.148291e+00,1.213322e+01,3.640865e+02,2.198834e+00,3.655942e+02,11433.567582,8.832031e+03,7.189082e+00,8679.003819,14.275584,1.542211,0.815164,1.91996,4.434959,24.014366,2.643092,4.937000,0.485074,0.0,0.485832,0.157306,1.999809,0.830046,2.064860,5.919048,6.652252,0.502779,5.258971,1.732251,2.272881,0.496283,20.157284,0.328984,0.227070,9347.389479,9.243830e+05,1.179858,1.250034,0.318741,2.130848e+01,1.694212e+00,1.428223e+00,1.084382e+01,1.809714e+01,1.553319e+01,3.683820e+02,6.943479e+01,3.717773e+02,7.005726e+01,0.0,0.0,5.538984e+00,1.809714e+01,1.553319e+01
min,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,-7.398828e+02,2.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,55.000000,2.941000e+03,0.000000e+00,2941.000000,0.000000,-3.138562,1.000000,61.00000,37.000000,9.000000,37.000000,4.000000,10.000000,9.0,8.000000,0.000000,37.000000,89.000000,96.000000,61.000000,25.000000,42.000000,4.000000,104.000000,35.000000,45.000000,1.000000,5.000000,102.000000,2948.000000,7.430000e+03,1.000000,19.000000,5.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,1.100000e+01,1.000000e+00,1.100000e+01,2.0,2.0,1.000000e+00,2.000000e+00,1.000000e+00
25%,8.990000e+02,9.000000e+02,1.000000e+00,2.100000e+01,1.400000e+01,4.700000e+01,3.653680e-01,3.000000e+01,2.170000e+02,1.000000e+00,2.170000e+02,433.000000,5.216000e+03,5.000000e+00,5216.000000,11.313708,-1.162647,1.000000,63.00000,38.0

In [19]:
"""
# Plot visualisation of the missing values for each feature of the raw DataFrame, df_sb
msno.matrix(df_sb, figsize = (30, 7))
"""

'\n# Plot visualisation of the missing values for each feature of the raw DataFrame, df_sb\nmsno.matrix(df_sb, figsize = (30, 7))\n'

In [20]:
# Counts of missing values
null_value_stats = df_sb.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

duration                           844164
tactics_formation                 3154857
tactics_lineup                    3154857
related_events                     134286
location                            24733
                                   ...   
home_team_managers                 340420
away_team_away_team_group         2841295
away_team_managers                 340420
metadata_shot_fidelity_version     663664
metadata_xy_fidelity_version      1003820
Length: 143, dtype: int64

The visualisation shows us that there are no missing values in the DataFrame.

---

## <a id='#section4'>4. Data Engineering</a>
Before conducting an [Exploratory Data Analysis (EDA)](#section5) of the data, we'll first need to clean and wrangle the datasets to a form that meet our needs.

### <a id='#section4.1'>4.1. Sort DataFrame</a>
Sort data by `match_date`, `kick_off`, and `period`, `timestamp`, as well as a few other attributes to be sure. Important for when determining previous events. which are attributes created for the DataFrame in the Data Engineering notebook.

In [26]:
# Sort data by matchId, matchPeriod, and eventSec
df_sb = df_sb.sort_values(['match_date', 'kick_off', 'competition_name', 'season_name', 'home_team_home_team_name', 'away_team_away_team_name', 'period', 'timestamp', 'index_x'])

In [27]:
df_sb.head(10)

,level_0,id,index_x,period,timestamp,minute,second,possession,duration,type_id,type_name,possession_team_id,possession_team_name,play_pattern_id,play_pattern_name,team_id,team_name,tactics_formation,tactics_lineup,related_events,location,player_id,player_name,position_id,position_name,pass_recipient_id,pass_recipient_name,pass_length,pass_angle,pass_height_id,pass_height_name,pass_end_location,pass_type_id,pass_type_name,pass_body_part_id,pass_body_part_name,carry_end_location,pass_outcome_id,pass_outcome_name,under_pressure,clearance_head,clearance_body_part_id,clearance_body_part_name,counterpress,duel_outcome_id,duel_outcome_name,duel_type_id,duel_type_name,ball_receipt_outcome_id,ball_receipt_outcome_name,out,clearance_left_foot,pass_switch,off_camera,clearance_aerial_won,dribble_outcome_id,dribble_outcome_name,pass_cross,pass_assisted_shot_id,pass_shot_assist,shot_statsbomb_xg,shot_end_location,shot_key_pass_id,shot_body_part_id,shot_body_part_name,shot_technique_id,shot_technique_name,shot_outcome_id,shot_outcome_name,shot_type_id,shot_type_name,shot_freeze_frame,goalkeeper_end_location,goalkeeper_type_id,goalkeeper_type_name,goalkeeper_position_id,goalkeeper_position_name,ball_recovery_recovery_failure,foul_committed_advantage,foul_won_advantage,dribble_overrun,clearance_right_foot,interception_outcome_id,interception_outcome_name,foul_won_defensive,pass_aerial_won,pass_deflected,pass_inswinging,pass_technique_id,pass_technique_name,goalkeeper_body_part_id,goalkeeper_body_part_name,goalkeeper_technique_id,goalkeeper_technique_name,goalkeeper_outcome_id,goalkeeper_outcome_name,pass_outswinging,pass_goal_assist,shot_one_on_one,miscontrol_aerial_won,shot_deflected,block_deflection,shot_first_time,block_offensive,pass_through_ball,foul_committed_card_id,foul_committed_card_name,foul_committed_penalty,foul_won_penalty,dribble_nutmeg,pass_miscommunication,pass_no_touch,foul_committed_offensive,goalkeeper_lost_out,pass_straight,substitution_outcome_id,substitution_outcome_name,substitution_replacement_id,substitution_replacement_name,match_id,goalkeeper_punched_out,shot_aerial_won,pass_cut_back,goalkeeper_success_in_play,50_50_outcome_id,50_50_outcome_name,foul_committed_type_id,foul_committed_type_name,ball_recovery_offensive,shot_saved_off_target,goalkeeper_shot_saved_off_target,shot_open_goal,dribble_no_touch,bad_behaviour_card_id,bad_behaviour_card_name,half_start_late_video_start,block_save_block,shot_follows_dribble,clearance_other,goalkeeper_shot_saved_to_post,shot_redirect,injury_stoppage_in_chain,shot_saved_to_post,goalkeeper_success_out,goalkeeper_lost_in_play,half_end_early_video_end,player_off_permanent,goalkeeper_saved_to_post,pass_backheel,shot_kick_off,goalkeeper_penalty_saved_to_post,index_y,match_date,kick_off,home_score,away_score,match_status,last_updated,match_week,referee,stadium,competition_competition_id,competition_country_name,competition_competition_name,season_season_id,season_season_name,home_team_home_team_id,home_team_home_team_name,home_team_home_team_gender,home_team_home_team_group,home_team_managers,home_team_country_id,home_team_country_name,away_team_away_team_id,away_team_away_team_name,away_team_away_team_gender,away_team_away_team_group,away_team_managers,away_team_country_id,away_team_country_name,metadata_data_version,metadata_shot_fidelity_version,metadata_xy_fidelity_version,competition_stage_id,competition_stage_name,competition_id,season_id,country_name,competition_name,competition_gender,season_name,match_updated,match_available
65644,0,d78142c2-54c8-48fd-8346-67ab75d01a5e,1,1,00:00:00.000,0,0,1,0.000000,35,Starting XI,1,Arsenal,1,Regular Play,1,Arsenal,442.0,"[{'player': {'id': 20015, 'name': 'Jens Lehman...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

### <a id='#section4.3'>4.3. Create Engineered Attributes</a>

#### <a id='#section4.2.1'>4.2.1. Create `Team` and `Opponent` Attributes</a>

In [ ]:
df_sb['Team'] = np.where(df_sb['team.name'] == df_sb['home_team_home_team_name'], df_sb['home_team_home_team_name'], df_sb['away_team_away_team_name'])
df_sb['Opponent'] = np.where(df_sb['team.name'] == df_sb['away_team_away_team_name'], df_sb['home_team_home_team_name'], df_sb['away_team_away_team_name'])

#### <a id='#section4.2.2'>4.2.2. Create `Full_Fixture_Date` Attribute</a>

In [ ]:
df_sb['full_fixture_date'] = df_sb['match_date'].astype(str) + ' ' + df_sb['home_team.home_team_name'].astype(str)  + ' ' + df_sb['home_score'].astype(str) + ' ' + ' vs. ' + ' ' + df_sb['away_score'].astype(str) + ' ' + df_sb['away_team.away_team_name'].astype(str)

In [ ]:
df_sb.head()

### <a id='#section4.4'>4.4. Subset DataFrame</a>
Subset DataFrame into
- In-play Events only
- Lineups - Starting XI
- Tactical Changes
- Halves

In [ ]:
# List unique values in the df_sb['type.name'] column
df_sb['type.name'].unique()

#### <a id='#section4.4.1'>4.4.1. Isolate In-Play Events</a>
DataFrame of only player's actions i.e. removing line ups, halves, etc.

##### <a id='#section4.4.1.1'>4.4.1.1. Remove Non-Event rows</a>

In [ ]:
lst_events = ['Pass', 'Ball Receipt*', 'Carry', 'Duel', 'Miscontrol', 'Pressure', 'Ball Recovery', 'Dribbled Past', 'Dribble', 'Shot', 'Block', 'Goal Keeper', 'Clearance', 'Dispossessed', 'Foul Committed', 'Foul Won', 'Interception', 'Shield', 'Half End', 'Substitution', 'Tactical Shift', 'Injury Stoppage', 'Player Off', 'Player On', 'Offside', 'Referee Ball-Drop', 'Error']

In [ ]:
df_sb_events = df_sb[df_sb['type.name'].isin(lst_events)]

In [ ]:
df_sb_events.shape

##### <a id='#section4.4.1.2'>4.4.1.2. Break down all `location` attributes into seperate attribute for X, Y (and sometimes Z) coordinates</a>

In [ ]:
# Display all location columns
for col in df_sb_events.columns:
    if 'location' in col:
        print(col)

There are the following five 'location' attributes:
- `location`
- `pass.end_location`
- `carry.end_location`
- `shot.end_location`
- `goalkeeper.end_location`

From reviewing the official documentation [[link](https://statsbomb.com/stat-definitions/)], the five attributes have the following dimensionality:
- `location` [x, y]
- `pass.end_location` [x, y]
- `carry.end_location` [x, y]
- `shot.end_location` [x, y, z]
- `goalkeeper.end_location` [x, y]

In [ ]:
"""
# CURRENTLY NOT WORKING, NEED TO FIX

# Normalize 'shot.freeze_frame' avvtribute - see: https://stackoverflow.com/questions/52795561/flattening-nested-json-in-pandas-data-frame

## explode all columns with lists of dicts
df_sb_events_normalize = df_sb_events.apply(lambda x: x.explode()).reset_index(drop=True)

## list of columns with dicts
cols_to_normalize = ['shot.freeze_frame']

## if there are keys, which will become column names, overlap with excising column names. add the current column name as a prefix
normalized = list()

for col in cols_to_normalize:
    d = pd.json_normalize(df_sb_events_normalize[col], sep='_')
    d.columns = [f'{col}_{v}' for v in d.columns]
    normalized.append(d.copy())

## combine df with the normalized columns
df_sb_events_normalize = pd.concat([df_sb_events_normalize] + normalized, axis=1).drop(columns=cols_to_normalize)

## display(df_lineup_select_normalize)
df_sb_events_normalize.head(30)
"""

In [ ]:
#

##
df_sb_events['location'] = df_sb_events['location'].astype(str)
df_sb_events['pass.end_location'] = df_sb_events['pass.end_location'].astype(str)
df_sb_events['carry.end_location'] = df_sb_events['carry.end_location'].astype(str)
df_sb_events['shot.end_location'] = df_sb_events['shot.end_location'].astype(str)
df_sb_events['goalkeeper.end_location'] = df_sb_events['goalkeeper.end_location'].astype(str)
df_sb_events['shot.end_location'] = df_sb_events['shot.end_location'].astype(str)
#df_sb_events['shot.freeze_frame'] = df_sb_events['shot.freeze_frame'].astype(str)


##

###
df_sb_events['location'] = df_sb_events['location'].str.replace('[','')
df_sb_events['pass.end_location'] = df_sb_events['pass.end_location'].str.replace('[','')
df_sb_events['carry.end_location'] = df_sb_events['carry.end_location'].str.replace('[','')
df_sb_events['shot.end_location'] = df_sb_events['shot.end_location'].str.replace('[','')
df_sb_events['goalkeeper.end_location'] = df_sb_events['goalkeeper.end_location'].str.replace('[','')
#df_sb_events['shot.freeze_frame'] = df_sb_events['shot.freeze_frame'].str.replace('[','')

###
df_sb_events['location'] = df_sb_events['location'].str.replace(']','')
df_sb_events['pass.end_location'] = df_sb_events['pass.end_location'].str.replace(']','')
df_sb_events['carry.end_location'] = df_sb_events['carry.end_location'].str.replace(']','')
df_sb_events['shot.end_location'] = df_sb_events['shot.end_location'].str.replace(']','')
df_sb_events['goalkeeper.end_location'] = df_sb_events['goalkeeper.end_location'].str.replace(']','')
#df_sb_events['shot.freeze_frame'] = df_sb_events['shot.freeze_frame'].str.replace(']','')


## Break down each location attributes
df_sb_events['location_x'], df_sb_events['location_y'] = df_sb_events['location'].str.split(',', 1).str
df_sb_events['pass.end_location_x'], df_sb_events['pass.end_location_y'] = df_sb_events['pass.end_location'].str.split(',', 1).str
df_sb_events['carry.end_location_x'], df_sb_events['carry.end_location_y'] = df_sb_events['carry.end_location'].str.split(',', 1).str
df_sb_events['shot.end_location_x'], df_sb_events['shot.end_location_y'], df_sb_events['shot.end_location_z'] = df_sb_events['shot.end_location'].str.split(',', 3).str[0:3].str
df_sb_events['goalkeeper.end_location_x'], df_sb_events['goalkeeper.end_location_y'] = df_sb_events['goalkeeper.end_location'].str.split(',', 1).str
#df_sb_events['shot.freeze_frame_x'], df_sb_events['shot.freeze_frame_y'] = df_sb_events['shot.freeze_frame'].str.split(',', 1).str


## Display DataFrame
df_sb_events.head(10)

In [ ]:
df_sb_events.shape

##### Export Dataset

In [ ]:
# Export 
#df_sb_events.to_csv(data_dir_sb + '/events/engineered/' + '/sb_events_1819_2021_wsl.csv', index=None, header=True)

# Export 
#df_sb_events.to_csv(data_dir + '/export/' + '/sb_wsl_events.csv', index=None, header=True)

##### <a id='#section4.4.1.3'>4.4.1.3. Create Passing Matrix Data</a>
The following DataFrame is the CSV extract used for Tableau dashboarding

In [ ]:
df1 = df_sb_events.copy()

In [ ]:
df1['df_name'] = 'df1'

In [ ]:
df1.head()

In [ ]:
df2 = df_sb_events.copy()

In [ ]:
df2['df_name'] = 'df2'

In [ ]:
df2.head()

In [ ]:
df1.head()

##### Concatanate DataFrames

In [ ]:
df_sb_events_passing = pd.concat([df1, df2])

In [ ]:
df_sb_events_passing.shape

##### ...

In [ ]:
df_sb_events_passing['Pass_X'] = np.where(df_sb_events_passing['df_name'] == 'df1', df_sb_events_passing['location_x'], df_sb_events_passing['pass.end_location_x'])
df_sb_events_passing['Pass_Y'] = np.where(df_sb_events_passing['df_name'] == 'df1', df_sb_events_passing['location_y'], df_sb_events_passing['pass.end_location_y'])

In [ ]:
df_sb_events_passing.head()

In [ ]:
sorted(df_sb_events_passing.columns)

##### Export Dataset

In [ ]:
# Export DataFrame as a CSV file
if not os.path.exists(os.path.join(data_dir_sb, 'export', 'sb_wsl_events_passing_matrix.csv')):
    df_sb_events_passing.to_csv(os.path.join(data_dir_sb, 'export', 'sb_wsl_events_passing_matrix.csv'), index=None, header=True)
else:
    pass

##### <a id='#section4.4.1.4'>4.4.1.4. Create Passing Network Data</a>

See: https://community.tableau.com/s/question/0D54T00000C6YbE/football-passing-network

In [ ]:
df_sb_pass_network = df_sb_events_passing.copy()

In [ ]:
df_sb_pass_network = df_sb_pass_network[df_sb_pass_network['type.name'] == 'Pass']

In [ ]:
df_sb_pass_network['player_recipient'] = np.where(df_sb_pass_network['df_name'] == 'df1', df_sb_pass_network['player.name'], df_sb_pass_network['pass.recipient.name'])

In [ ]:
df_sb_pass_network.head()

In [ ]:
sorted(df_sb_pass_network.columns)

In [ ]:
df_sb_pass_network.shape

In [ ]:
# Select columns of interest

## Define columns
cols = ['df_name',
        'id',
        'index',
        'competition_name',
        'season_name',
        'match_date',
        'kick_off',
        'Full_Fixture_Date',
        'Team',
        'Opponent',
        'home_team.home_team_name',
        'away_team.away_team_name',
        'home_score',
        'away_score',
        'player_recipient',
        'player.name',
        'pass.recipient.name',
        'position.id',
        'position.name',
        'type.name',
        'pass.type.name',
        'pass.outcome.name',
        'location_x',
        'location_y', 
        'pass.end_location_x',
        'pass.end_location_y',
        'Pass_X',
        'Pass_Y'
       ]

##
df_sb_pass_network_select = df_sb_pass_network[cols]

In [ ]:
df_sb_pass_network_select['pass.to.from'] = df_sb_pass_network_select['player.name'] + ' - ' + df_sb_pass_network_select['pass.recipient.name']

In [ ]:
# List unique values in the df_sb_pass_network_select['pass.outcome.name'] column
df_sb_pass_network_select['pass.outcome.name'].unique()

In [ ]:
df_sb_pass_network_select = df_sb_pass_network_select[df_sb_pass_network_select['pass.outcome.name'].isnull()]

In [ ]:
df_sb_pass_network_select.shape

In [ ]:
df_sb_pass_network_select = df_sb_pass_network_select.sort_values(['season_name', 'match_date', 'kick_off', 'Full_Fixture_Date', 'index', 'id', 'df_name'], ascending=[True, True, True, True, True, True, True])

In [ ]:
df_sb_pass_network_select['Pass_X'] = df_sb_pass_network_select['Pass_X'].astype(str).astype(float)
df_sb_pass_network_select['Pass_Y'] = df_sb_pass_network_select['Pass_Y'].astype(str).astype(float)
df_sb_pass_network_select['location_x'] = df_sb_pass_network_select['location_x'].astype(str).astype(float)
df_sb_pass_network_select['location_y'] = df_sb_pass_network_select['location_y'].astype(str).astype(float)
df_sb_pass_network_select['pass.end_location_x'] = df_sb_pass_network_select['pass.end_location_x'].astype(str).astype(float)
df_sb_pass_network_select['pass.end_location_y'] = df_sb_pass_network_select['pass.end_location_y'].astype(str).astype(float)

In [ ]:
df_sb_pass_network_select.dtypes

In [ ]:
df_sb_pass_network_select.head()

In [ ]:
#

##
df_sb_pass_network_grouped = (df_sb_pass_network_select
                                  .groupby(['competition_name',
                                            'season_name',
                                            'match_date',
                                            'kick_off',
                                            'Full_Fixture_Date',
                                            'Team',
                                            'Opponent',
                                            'home_team.home_team_name',
                                            'away_team.away_team_name',
                                            'home_score',
                                            'away_score',
                                            'pass.to.from',
                                            'player.name',
                                            'pass.recipient.name',
                                            'player_recipient'
                                           ])
                                  .agg({'pass.to.from': ['count']
                                       })
                             )

##
df_sb_pass_network_grouped.columns = df_sb_pass_network_grouped.columns.droplevel(level=0)

##
df_sb_pass_network_grouped = df_sb_pass_network_grouped.reset_index()

## 
df_sb_pass_network_grouped.columns = ['competition_name',
                                      'season_name',
                                      'match_date',
                                      'kick_off',
                                      'full_fixture_date',
                                      'team',
                                      'opponent',
                                      'home_team_name',
                                      'away_team_name',
                                      'home_score',
                                      'away_score',
                                      'pass_to_from',
                                      'player_name',
                                      'pass_recipient_name',
                                      'player_recipient',
                                      'count_passes',
                                     ]

##
#df_sb_pass_network_grouped['count_passes'] = df_sb_pass_network_grouped['count_passes'] / 2

##
df_sb_pass_network_grouped = df_sb_pass_network_grouped.sort_values(['season_name', 'match_date', 'kick_off', 'full_fixture_date', 'team', 'opponent', 'pass_to_from'], ascending=[True, True, True, True, True, True, True])

##
df_sb_pass_network_grouped.head()

In [ ]:
df_sb_pass_network_grouped.shape

In [ ]:
# Select columns of interest

## Define columns
cols = ['Full_Fixture_Date',
        'player.name',
        'position.id',
        'position.name',
        'Pass_X',
        'Pass_Y'
       ]

##
df_sb_pass_network_avg_pass = df_sb_pass_network_select[cols]

In [ ]:
df_sb_pass_network_avg_pass 

In [ ]:
#

##
df_sb_pass_network_avg_pass_grouped = (df_sb_pass_network_avg_pass 
                                          .groupby(['Full_Fixture_Date',
                                                    'player.name',
                                                    'position.id',
                                                    'position.name',
                                                   ])
                                          .agg({'Pass_X': ['mean'],
                                                'Pass_Y': ['mean']
                                               })
                                     )

##
df_sb_pass_network_avg_pass_grouped.columns = df_sb_pass_network_avg_pass_grouped .columns.droplevel(level=0)

##
df_sb_pass_network_avg_pass_grouped = df_sb_pass_network_avg_pass_grouped.reset_index()

## 
df_sb_pass_network_avg_pass_grouped.columns = ['full_fixture_date',
                                               'player_name',
                                               'position_id',
                                               'position_name',
                                               'avg_location_pass_x',
                                               'avg_location_pass_y'
                                     ]

##
df_sb_pass_network_avg_pass_grouped['avg_location_pass_x'] = df_sb_pass_network_avg_pass_grouped['avg_location_pass_x'].round(decimals=1)
df_sb_pass_network_avg_pass_grouped['avg_location_pass_y'] = df_sb_pass_network_avg_pass_grouped['avg_location_pass_y'].round(decimals=1)

##
df_sb_pass_network_avg_pass_grouped = df_sb_pass_network_avg_pass_grouped.sort_values(['full_fixture_date', 'player_name'], ascending=[True, True])

##
df_sb_pass_network_avg_pass_grouped.head()

In [ ]:
# Join the Events DataFrame to the Matches DataFrame
df_sb_pass_network_final = pd.merge(df_sb_pass_network_grouped, df_sb_pass_network_avg_pass_grouped, left_on=['full_fixture_date', 'player_recipient'], right_on=['full_fixture_date', 'player_name'])

In [ ]:
## Rename columns
df_sb_pass_network_final = df_sb_pass_network_final.rename(columns={'player_name_x': 'player_name',
                                                                   #'player_name_x': 'player_name'
                                                                   }
                                                          )

In [ ]:
df_sb_pass_network_final.head()

In [ ]:
df_sb_pass_network_final.shape

##### Export Dataset

In [ ]:
# Export DataFrame as a CSV file
if not os.path.exists(os.path.join(data_dir_sb, 'export', 'engineered', 'sb_events_passing_network.csv')):
    df_sb_pass_network_final.to_csv(os.path.join(data_dir_sb, 'export', 'engineered', 'sb_events_passing_network.csv'), index=None, header=True)
else:
    pass

In [ ]:
# Export DataFrame as a CSV file
if not os.path.exists(os.path.join(data_dir_sb, 'export', 'engineered', 'sb_events_passing_network.csv')):
    df_sb_pass_network_final.to_csv(os.path.join(data_dir, 'export', 'sb_events_passing_network.csv'), index=None, header=True)
else:
    pass

##### Export WSL data Dataset

In [ ]:
# Export 
#df_sb_pass_network_final.to_csv(data_dir_sb + '/events/engineered/' + '/sb_events_passing_network_1819_2021_wsl.csv', index=None, header=True)

# Export 
#df_sb_pass_network_final.to_csv(data_dir + '/export/' + '/sb_wsl_events_passing_network.csv', index=None, header=True)

#### <a id='#section4.4.2'>4.4.2. Lineups</a>

In [ ]:
# List unique values in the df_sb['type.name'] column
df_sb['type.name'].unique()

The starting XI players and formation can be found in the rows where `type.name` is 'Starting XI'.

In [ ]:
df_lineup = df_sb[df_sb['type.name'] == 'Starting XI']

In [ ]:
df_lineup

In [ ]:
# Streamline DataFrame to include just the columns of interest

## Define columns
cols = ['id', 'type.name', 'match_date', 'kick_off', 'Full_Fixture_Date', 'team.id', 'team.name', 'tactics.formation', 'tactics.lineup', 'competition_name', 'season_name', 'home_team.home_team_name', 'away_team.away_team_name', 'Team', 'Opponent', 'home_score', 'away_score']

## Select only columns of interest
df_lineup_select = df_lineup[cols]

In [ ]:
df_lineup_select

We can see from the extracted lineup data so far. To get the stating XI players, we need to breakdown the `tactics.lineup` attribute.

In [ ]:
# Normalize tactics.lineup - see: https://stackoverflow.com/questions/52795561/flattening-nested-json-in-pandas-data-frame

## explode all columns with lists of dicts
df_lineup_select_normalize = df_lineup_select.apply(lambda x: x.explode()).reset_index(drop=True)

## list of columns with dicts
cols_to_normalize = ['tactics.lineup']

## if there are keys, which will become column names, overlap with excising column names. add the current column name as a prefix
normalized = list()

for col in cols_to_normalize:
    d = pd.json_normalize(df_lineup_select_normalize[col], sep='_')
    d.columns = [f'{col}_{v}' for v in d.columns]
    normalized.append(d.copy())

## combine df with the normalized columns
df_lineup_select_normalize = pd.concat([df_lineup_select_normalize] + normalized, axis=1).drop(columns=cols_to_normalize)

## display(df_lineup_select_normalize)
df_lineup_select_normalize.head(30)

In [ ]:
df_lineup_engineered = df_lineup_select_normalize

In [ ]:
# Streamline DataFrame to include just the columns of interest

## Define columns
cols = ['id', 'match_date', 'kick_off', 'Full_Fixture_Date', 'type.name', 'season_name', 'competition_name', 'home_team.home_team_name', 'away_team.away_team_name', 'Team', 'Opponent', 'home_score', 'away_score', 'tactics.formation', 'tactics.lineup_jersey_number', 'tactics.lineup_position_id', 'tactics.lineup_player_name', 'tactics.lineup_position_name']

## Select only columns of interest
df_lineup_engineered_select = df_lineup_engineered[cols]

In [ ]:
df_lineup_engineered_select['tactics.formation'] = df_lineup_engineered_select['tactics.formation'].astype('Int64')
df_lineup_engineered_select['tactics.lineup_jersey_number'] = df_lineup_engineered_select['tactics.lineup_jersey_number'].astype('Int64')

In [ ]:
df_lineup_engineered_select.head(5)

In [ ]:
df_lineup_engineered_select.columns

In [ ]:
## Rename columns
df_lineup_engineered_select = df_lineup_engineered_select.rename(columns={'id': 'Match_Id',
                                                                          'match_date': 'Match_Date',
                                                                          'kick_off': 'Kick_Off',
                                                                          'type.name': 'Type_Name',
                                                                          'season_name': 'Season',
                                                                          'competition_name': 'Competition',
                                                                          'home_team.home_team_name': 'Home_Team',
                                                                          'away_team.away_team_name': 'Away_Team',
                                                                          'home_score': 'Home_Score',
                                                                          'away_score': 'Away_Score',
                                                                          'tactics.formation': 'Formation',
                                                                          'tactics.lineup_jersey_number': 'Shirt_Number',
                                                                          'tactics.lineup_position_id': 'Position_Number',
                                                                          'tactics.lineup_player_name': 'Player_Name',
                                                                          'tactics.lineup_position_name': 'Position_Name'
                                                                         }
                                                                         
                                                                )

## Display DataFrame
df_lineup_engineered_select.head()

In [ ]:
# Convert Match_Date from string to datetime64[ns]
df_lineup_engineered_select['Match_Date']= pd.to_datetime(df_lineup_engineered_select['Match_Date'])

In [ ]:
"""
# THIS IS NOT WORKING ATM

# Convert Kick_Off from string to datetime64[ns]
df_lineup_engineered_select['Kick_Off']= pd.to_datetime(df_lineup_engineered_select['Kick_Off'], format='%H:%M', errors='ignore')
df_lineup_engineered_select['Kick_Off'] = df_lineup_engineered_select['Kick_Off'].dt.time
"""

In [ ]:
df_lineup_engineered_select.dtypes

In [ ]:
# Put hyphens between numbers in Formation attribute

## Convert Formation attribute from Integer to String
df_lineup_engineered_select['Formation'] = df_lineup_engineered_select['Formation'].astype(str)

## Define custom function to add hyphen between letters: StackOverflow: https://stackoverflow.com/questions/29382285/python-making-a-function-that-would-add-between-letters
def f(s):
        m = s[0]
        for i in s[1:]:
             m += '-' + i
        return m
    
## Apply custom function
df_lineup_engineered_select['Formation'] = df_lineup_engineered_select.apply(lambda row: f(row['Formation']),axis=1)

In [ ]:
lst_formation = df_lineup_engineered_select['Formation'].unique().tolist()

In [ ]:
lst_formation

##### Add Position Coordinates

In [ ]:
df_formations_coords = pd.read_csv(data_dir_sb + '/sb_formation_coordinates.csv')

In [ ]:
#df_formations_coords['Id'] = df_formations_coords['Id'].astype('Int8')
#df_formations_coords['Player_Number'] = df_formations_coords['Player_Number'].astype('Int8')

In [ ]:
df_lineup_engineered_select = pd.merge(df_lineup_engineered_select, df_formations_coords, how='left', left_on=['Formation', 'Position_Number'], right_on=['Formation', 'Player_Number'])

In [ ]:
#df_lineup_engineered_select = df_lineup_engineered_select.drop(['Player_Number'], axis=1)
df_lineup_engineered_select = df_lineup_engineered_select.drop(['Id'], axis=1)
df_lineup_engineered_select = df_lineup_engineered_select.drop(['Player_Position'], axis=1)

In [ ]:
df_lineup_engineered_select.head()

##### Add Opponent Data to Each Row

In [ ]:
# Select columns of interest

## Define columns
cols = ['Match_Date',
        'Competition',
        'Full_Fixture_Date',
        'Team',
        'Formation'
       ]

##
df_lineup_opponent = df_lineup_engineered_select[cols]

##
df_lineup_opponent = df_lineup_opponent.drop_duplicates()

##
df_lineup_opponent.head()

In [ ]:
# Join DataFrame to itself on 'Date', 'Fixture', 'Team'/'Opponent', and 'Event', to join Team and Opponent together
df_lineup_engineered_opponent_select = pd.merge(df_lineup_engineered_select, df_lineup_opponent,  how='left', left_on=['Match_Date', 'Competition', 'Full_Fixture_Date', 'Opponent'], right_on = ['Match_Date', 'Competition', 'Full_Fixture_Date', 'Team'])

In [ ]:
# Clean Data

## Drop columns
df_lineup_engineered_opponent_select = df_lineup_engineered_opponent_select.drop(columns=['Team_y'])


## Rename columns
df_lineup_engineered_opponent_select = df_lineup_engineered_opponent_select.rename(columns={'Team_x': 'Team',
                                                                                            'Formation_x': 'Formation',
                                                                                            'Formation_y': 'Opponent_Formation'
                                                                                           }
                                                                                      )

## Display DataFrame
df_lineup_engineered_opponent_select.head()

##### Export DataFrame

In [ ]:
# Export 
#df_lineup_engineered_opponent_select.to_csv(data_dir_sb + '/lineups/engineered/' + '/sb_lineups_1819_2021_wsl.csv', index=None, header=True)

In [ ]:
# Export 
#df_lineup_engineered_opponent_select.to_csv(data_dir + '/export/' + '/sb_wsl_lineups.csv', index=None, header=True)

#### <a id='#section4.4.3'>4.4.3. Tactical Shifts</a>

In [ ]:
df_tactics = df_sb[df_sb['type.name'] == 'Tactical Shift']

In [ ]:
df_tactics

In [ ]:
# Select columns of interest

##
cols = ['id', 'type.name', 'team.id', 'team.name', 'tactics.formation', 'tactics.lineup']

##
df_tactics_select = df_tactics[cols]

In [ ]:
df_tactics_select

In [ ]:
# Normalize tactics.lineup - see: https://stackoverflow.com/questions/52795561/flattening-nested-json-in-pandas-data-frame

## explode all columns with lists of dicts
df_tactics_select_normalize = df_tactics_select.apply(lambda x: x.explode()).reset_index(drop=True)

## list of columns with dicts
cols_to_normalize = ['tactics.lineup']

## if there are keys, which will become column names, overlap with excising column names. add the current column name as a prefix
normalized = list()
for col in cols_to_normalize:
    
    d = pd.json_normalize(df_tactics_select_normalize[col], sep='_')
    d.columns = [f'{col}_{v}' for v in d.columns]
    normalized.append(d.copy())

## combine df with the normalized columns
df_tactics_select_normalize = pd.concat([df_tactics_select_normalize] + normalized, axis=1).drop(columns=cols_to_normalize)

## display(df_lineup_select_normalize)
df_tactics_select_normalize.head(10)

#### <a id='#section4.4.4'>4.4.4. Halves</a>

In [ ]:
df_half = df_sb[df_sb['type.name'] == 'Half Start']

In [ ]:
df_half

---

## <a id='#section5'>5. Export Data</a>
Export Data ready for data engineering in the subsequent notebooks.

In [ ]:
# Export 
#df_sb.to_csv(data_dir_sb + '/combined/raw/csv/wsl/' + '/df_sb_combined_data_wsl.csv', index=None, header=True)

## <a id='#section6'>6. Summary</a>
This notebook engineers scraped [StatsBomb](https://statsbomb.com/) data using [pandas](http://pandas.pydata.org/) for data manipulation through DataFrames.

---

## <a id='#section7'>7. Next Steps</a>
The step is to take the parsed dataset created in this notebook and engineer the data for new features, which is carried out in the follow [Data Engineering](https://nbviewer.jupyter.org/github/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/StatsBomb%20Data%20Engineering.ipynb) notebook. This data is then ready for use in projects including Expected Goals (xG) models and Tableau visualisations.

## <a id='#section8'>8. References</a>

#### Data
*    [StatsBomb](https://statsbomb.com/) data
*    [StatsBomb](https://github.com/statsbomb/open-data/tree/master/data) open data GitHub repository

---

***Visit my website [EddWebster.com](https://www.eddwebster.com) or my [GitHub Repository](https://github.com/eddwebster) for more projects. If you'd like to get in contact, my Twitter handle is [@eddwebster](http://www.twitter.com/eddwebster) and my email is: edd.j.webster@gmail.com.***

[Back to the top](#top)